In [31]:
# import libraries
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from geopy.geocoders import Nominatim
from translate import Translator
import seaborn as sns
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [32]:
root_path = "C:\\Users\\Solar\\Furiends\\Furiends\\data\\raw\\Fake Data2\\bigT3.csv"

In [33]:
data = pd.read_csv(root_path)

In [34]:
data

,User_id,First_Name,Last_Name,Address,Age,Gender,Rehabilitation,Pet_intimacy,category_name,Pet_Name,log_count,com_count,avg_loading,sum_duration
0,1,Jane,Bruen,5272 Adela Overpass Suite 982\r\nRozellacheste...,28,1,2,4.2,'Maltese',Barney,5,5,2611.97,70.8
1,2,Eliseo,Trantow,"2506 Mertz Inlet\r\nSouth Ezekielshire, OK 56215",12,1,1,1.2,'Golden Retriever',Juliana,1,2,4931.20,4.1
2,3,Oceane,Walter,"9469 Frank Orchard Apt. 685\r\nLake Irmastad, ...",64,1,2,9.1,'Bulldog',Sabrina,7,3,2610.35,59.9
3,4,Zack,Marvin,"982 Susan Manors\r\nPort Amiya, UT 97043-2723",38,1,1,8.6,'German Shepherd',Yasmine,9,8,2727.02,212.3
4,5,Marco,Kessler,"671 Troy Isle Suite 459\r\nNew Carriemouth, OK...",28,1,1,1.9,'German Shepherd',Cristopher,4,7,4054.30,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,Ethan,Schuppe,"17224 Koelpin Rest\r\nTheresiashire, GA 69914",79,2,1,8.2,'Bulldog',Filomena,4,5,1971.06,80.3
596,597,Austin,Legros,"476 Veda View\r\nGroverfurt, AK 15766-2282",80,1,2,9.1,'Shi-ba-inu',Herta,3,7,2113.40,53.9
597,598,Ilene,Dietrich,"934 Josianne Inlet\r\nLavonnechester, AZ 30057...",74,1,2,2.8,'Poodle',Lamont,2,5,1083.90,4.3
598,599,Novella,Becker,"76281 Germaine Valley\r\nBoscobury, DE 99069-3814",29,2,2,1.0,'Golden Retriever',Wendy,7,3,2126.39,193.7


In [35]:
data.columns

Index(['User_id', 'First_Name', 'Last_Name', 'Address', 'Age', 'Gender',
       'Rehabilitation', 'Pet_intimacy', 'category_name', 'Pet_Name',
       'log_count', 'com_count', 'avg_loading', 'sum_duration'],
      dtype='object')

In [36]:
data.shape

(600, 14)

## 1. Check Numeric Data Range

In [37]:
data.loc[:, "Age"].describe()

count    600.000000
mean      49.863333
std       22.283527
min       12.000000
25%       30.000000
50%       50.000000
75%       69.000000
max       90.000000
Name: Age, dtype: float64

Oldest user age: 90 <br>
Yougest user age: 12<br>

In [38]:
data.loc[:, "Pet_intimacy"].describe()

count    600.000000
mean       4.875000
std        2.853708
min        0.000000
25%        2.500000
50%        4.850000
75%        7.300000
max       10.000000
Name: Pet_intimacy, dtype: float64

In [39]:
# replace all intimacy = 0 to 1
data["Pet_intimacy"] = data["Pet_intimacy"].replace(0, 1)
data.loc[:, "Pet_intimacy"].describe()

count    600.000000
mean       4.878333
std        2.848582
min        0.100000
25%        2.500000
50%        4.850000
75%        7.300000
max       10.000000
Name: Pet_intimacy, dtype: float64

minimum intimacy: 0<br>
maxium intimacy: 10

In [40]:
sum(data["avg_loading"] > data["sum_duration"])

596

What's average loading meaning? <br>
Does it relate to sum duration? <br>
The average loading of 4 data is less than their sum duration, is this a problem?

## 2. Check Categorical Data Value

In [41]:
data.loc[:, "Gender"].value_counts()

1    304
2    296
Name: Gender, dtype: int64

In [42]:
data.loc[:, "Rehabilitation"].value_counts()

2    300
1    300
Name: Rehabilitation, dtype: int64

In [43]:
data.loc[:,"category_name"].value_counts()

'Bulldog'             115
'Maltese'             102
'Poodle'               99
'German Shepherd'      97
'Golden Retriever'     94
'Shi-ba-inu'           93
Name: category_name, dtype: int64

## 3. Address Split

In [44]:
def get_continent(col):
    try:
        col = col.strip()
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)


def get_country(zipcode):
    try:
        country = geolocator.geocode(zipcode, language='en').address.split(',')[-1]
        return country
    except AttributeError:
        return None

In [45]:
# create a new column named zipcode and grab zip code information from address
data["Zipcode"] = data["Address"].str.split(", ").str.get(-1).str.split(" ").str.get(-1).str[:5]

# get the country name using zipcode
geolocator = Nominatim(user_agent="geoapiExercises")
data["Country"] = data.loc[:, "Zipcode"].apply(lambda x: get_country(x))

# delete data with invalid zipcode (zipcode that could not find a corresponding location)
data = data.loc[data.loc[:,"Country"].str.isnumeric() == False]

# create columns to store the abbreviate country name and country's continent name
data["Country_abbrev"] = data.loc[:, "Country"].apply(lambda x: get_continent(x)[0])
data["Continent"] = data.loc[:, "Country"].apply(lambda x: get_continent(x)[1])

C:\Users\Solar\AppData\Local\Temp\ipykernel_9324\3356182490.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Country_abbrev"] = data.loc[:, "Country"].apply(lambda x: get_continent(x)[0])
C:\Users\Solar\AppData\Local\Temp\ipykernel_9324\3356182490.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Continent"] = data.loc[:, "Country"].apply(lambda x: get_continent(x)[1])


In [46]:
data

,User_id,First_Name,Last_Name,Address,Age,Gender,Rehabilitation,Pet_intimacy,category_name,Pet_Name,log_count,com_count,avg_loading,sum_duration,Zipcode,Country,Country_abbrev,Continent
0,1,Jane,Bruen,5272 Adela Overpass Suite 982\r\nRozellacheste...,28,1,2,4.2,'Maltese',Barney,5,5,2611.97,70.8,14374,United States,US,NA
1,2,Eliseo,Trantow,"2506 Mertz Inlet\r\nSouth Ezekielshire, OK 56215",12,1,1,1.2,'Golden Retriever',Juliana,1,2,4931.20,4.1,56215,South Korea,KR,AS
2,3,Oceane,Walter,"9469 Frank Orchard Apt. 685\r\nLake Irmastad, ...",64,1,2,9.1,'Bulldog',Sabrina,7,3,2610.35,59.9,72714,United States,US,NA
3,4,Zack,Marvin,"982 Susan Manors\r\nPort Amiya, UT 97043-2723",38,1,1,8.6,'German Shepherd',Yasmine,9,8,2727.02,212.3,97043,Ukraine,UA,EU
4,5,Marco,Kessler,"671 Troy Isle Suite 459\r\nNew Carriemouth, OK...",28,1,1,1.9,'German Shepherd',Cristopher,4,7,4054.30,14.6,61216,South Korea,KR,AS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,Ethan,Schuppe,"17224 Koelpin Rest\r\nTheresiashire, GA 69914",79,2,1,8.2,'Bulldog',Filomena,4,5,1971.06,80.3,69914,Brazil,BR,SA
596,597,Austin,Legros,"476 Veda View\r\nGroverfurt, AK 15766-2282",80,1,2,9.1,'Shi-ba-inu',Herta,3,7,2113.40,53.9,15766,Brazil,BR,SA
597,598,Ilene,Dietrich,"934 Josianne Inlet\r\nLavonnechester, AZ 30057...",74,1,2,2.8,'Poodle',Lamont,2,5,1083.90,4.3,30057,Taiwan,TW,AS
598,599,Novella,Becker,"76281 Germaine Valley\r\nBoscobury, DE 99069-3814",29,2,2,1.0,'Golden Retriever',Wendy,7,3,2126.39,193.7,99069,United States,US,NA


## 4. Check for null values

In [47]:
for c in data.columns:
    if data[c].isnull().values.any():
        print(c)

avg_loading
sum_duration


In [48]:
data = data.dropna()

In [50]:
data.shape

(575, 18)

In [49]:
data.to_csv('data.csv')